# TitaniumCloud TAXII Related Tools Feed
This notebook demonstrates how to use the TCTF-0003 TAXII Related Tools Feed which delivers fresh and up-to-date indicators of ransomware activity in the wild.
**NOTE:** If pasted into a Python file in the displayed order, all code cells can also work as a Python script.

### Covered API classes
This notebook covers examples for the following API class:
- **TAXIIFeed** (*TCTF-0003*)

### Credentials
Credentials are loaded from a local file instead of being written here in plain text.
To learn how to creat the credentials file, see the **Storing and using the credentials** section in the [README file](./README.md)

### 1. Importing the required classes
First, we will import the required API class from the ticloud module.

In [ ]:
from ReversingLabs.SDK.ticloud import TAXIIFeed

### 2. Loading the credentials
Next, we will load our TitaniumCloud credentials from the local `ticloud_credentials.json` file.
**NOTE: Instead of doing this step, you can paste your credentials while creating the Python object in the following step.**

In [ ]:
import json


CREDENTIALS = json.load(open("ticloud_credentials.json"))
USERNAME = CREDENTIALS.get("username")
PASSWORD = CREDENTIALS.get("password")
USER_AGENT = json.load(open('../user_agent.json'))["user_agent"]

### 3. Creating the TAXIIFeed object
Now we need to create the Python object for the TAXIIFeed in order to use its quota displaying methods.

In [ ]:
taxii_feed = TAXIIFeed(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent=USER_AGENT
)

### 4. Standard endpoints
Every TAXII standard server has the following endpoints:
- Discovery endpoint
- API root endpoint
- Collections endpoint

#### Discovery endpoint
The discovery endpoint is the starting point for learning about the available API roots on a TAXII server and to get the general info about the server. To fetch the discovery info from the TAXII Related Tools feed, do the run the following action:

In [ ]:
discovey_info = taxii_feed.discovery_info()
print(discovey_info.text)

Now that we have discovery info, we can see the available API roots.

#### API root endpoint
To get the info about a specific API root, run the following action:

In [ ]:
api_rooot_info = taxii_feed.api_root_info(api_root="flexible-intel-feeds")
print(api_rooot_info.text)

#### Collections endpoint
To get the info about a specific collection on an API root, run the following action:

In [ ]:
collection_info = taxii_feed.collections_info(api_root="flexible-intel-feeds")
print(collection_info.text)

### 5. Getting objects from the feed

#### Single page of objects
To fetch a single page of objects from a specified time onwards, do the following:

In [ ]:
from datetime import datetime, timedelta

hours_back = 5.0    
desired_time = datetime.today() - timedelta(hours=hours_back)
time_string = desired_time.strftime("%Y-%m-%dT%H:%M:%SZ")

one_page = taxii_feed.get_objects(
    api_root="flexible-intel-feeds",
    collection_id="049cf182-4812-5c6e-a539-5efc36c05aaa",
    added_after=time_string
)

print(one_page.text)

This example fetches data from the previous 5 hours relatively. To change the range, edit the `hours_back` variable.

#### Objects from multiple pages
Paging can be done manually or by using the `get_objects_aggregated` method which does the paging withing itself and returns a desired number of results. If the `max_results` parameter is set to `None`, ALL available results for the defined period will be returned.

In [ ]:
results_list = taxii_feed.get_objects_aggregated(
    api_root="flexible-intel-feeds",
    collection_id="049cf182-4812-5c6e-a539-5efc36c05aaa",
    added_after=time_string,
    result_limit=50,
    max_results=500
)

print(results_list)

In our example, we defined that we need a maximum of 500 results. In case there is less than 500 results available for the defined period, the list will return the available maximum. The aggregating method does the paging for us and lets us worry only about the maximum desired number of results and the number of results returned per each page (`result_limit` - affects the number of requests being done in the background).

#### Single specific object
Apart from fetching latest indicators in general, the TAXII feed also allows fetching a single specific indicator using its ID:

In [ ]:
specific_indicator = taxii_feed.get_objects(
    api_root="flexible-intel-feeds",
    collection_id="049cf182-4812-5c6e-a539-5efc36c05aaa",
    match_id="indicator--1e14a458-f3f8-5d26-8049-097e00e55aa2"
)

print(specific_indicator.text)